In [35]:
import pandas as pd
import random


In [2]:
results = pd.read_csv('results.csv')
roster = pd.read_csv('roster.csv')
tournaments = pd.read_csv('tournaments.csv')

In [3]:
results.head()

,idteam,current_name,base_name,position,questions_total,mask,bonus_a,bonus_b,tech_rating,predicted_position,d_bonus_a,d_bonus_b,d_diff_bonus,included_in_rating,idtournament,diff_bonus
0,209,Команда Коваленко,Команда Коваленко,11.0,34,1111101000101111001100011000111101001111001110...,2230.0,-87.0,4509.0,7.0,2230,663,-87.0,1.0,3236,NaN
1,270,Синоп,Синоп,14.5,31,1110110000101100001100101101101101010011010000...,2189.0,119.0,2567.0,20.0,2189,618,119.0,1.0,3236,NaN
2,358,43,43,20.0,26,1111110000011100000010011111100001000111001000...,2074.0,-100.0,3532.0,9.0,2074,501,-100.0,1.0,3236,NaN
3,482,Нимлот,Нимлот,14.5,31,1101100000001110001011011101111111101111100100...,2189.0,-29.0,3376.0,10.0,2189,618,-29.0,1.0,3236,NaN
4,1948,Doom-Doom,Doom-Doom,17.5,29,1111010010110110001000011001101100101001000000...,2145.0,38.0,2629.0,19.0,2145,557,38.0,1.0,3236,NaN


In [32]:
df = results.loc[results.idtournament == 3236, ['idteam', 'mask']].set_index('idteam')
df.head()

,mask
idteam,
209,1111101000101111001100011000111101001111001110...
270,1110110000101100001100101101101101010011010000...
358,1111110000011100000010011111100001000111001000...
482,1101100000001110001011011101111111101111100100...
1948,1111010010110110001000011001101100101001000000...


In [33]:
length = len(df.iloc[0, 0])
mask = df.loc[:, 'mask'].str.split('', length, expand=True).drop(columns=[0])
mask.head()

,1,2,3,4,5,6,7,8,9,10,...,63,64,65,66,67,68,69,70,71,72
idteam,,,,,,,,,,,,,,,,,,,,,
209,1,1,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
270,1,1,1,0,1,1,0,0,0,0,...,0,0,0,1,1,0,1,1,0,1
358,1,1,1,1,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
482,1,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1948,1,1,1,1,0,1,0,0,1,0,...,0,1,0,1,0,1,1,0,0,0


## Model

In [34]:
roster.head()

,idplayer,is_captain,is_base,is_foreign,idteam,idtournament
0,19298,1,1,0,209,3236
1,7474,0,1,0,209,3236
2,15624,0,1,0,209,3236
3,32017,0,1,0,209,3236
4,13601,0,1,0,209,3236


In [53]:
def skill_init():
    return random.uniform(0, 10)
player_skills_dict = {id: skill_init() for id in roster.idplayer}
pd.DataFrame(player_skills_dict, index=[0]).T.head()

,0
19298,1.127116
7474,4.043970
15624,1.958347
32017,8.120617
13601,9.301064


In [54]:
def difficulty_init():
    return random.uniform(0, 10)
question_difficulty = {ques: difficulty_init() for ques in mask.columns}
pd.DataFrame(question_difficulty, index=[0]).T.head()

,0
1,7.700699
2,1.098969
3,5.711775
4,4.976770
5,4.929899


In [55]:
def sigmoid(x):
    return 1 / (1 + pd.np.exp(-x))

In [64]:
def team_skill(team_id, tour_id):
    players = roster.loc[(roster.idteam == team_id) & (roster.idtournament == tour_id), 'idplayer'].tolist()
    skills = [player_skills_dict[player] for player in players]
    total_skill = sum(skills)
    return total_skill

team_skill(209, 3236)

28.122416274572338

In [66]:
def correct_answer_prob(team_id, tour_id, ques_id):
    difficulty = question_difficulty[ques_id]
    t_skill = team_skill(team_id, tour_id)
    prob = sigmoid(t_skill - difficulty)
    return prob

correct_answer_prob(209, 3236, 1)

0.9999999986480499